# Автолекс

In [1]:
from CFG import CFG
from RG import RG
from Rule import Rule
from Token import Token
from PumpTree import PumpTree
from Test import Test

from utils import add_method, Set
import os
from copy import copy

## Обертка в охранные нетерминалы

In [3]:
def guard(cfg):
    previous_cfg = copy(cfg)
    regular_nterms = previous_cfg.regular_nterms()
    new_rules = []
    for nterm in previous_cfg.nterms:
        if nterm in regular_nterms:
            continue

        for rule in previous_cfg[nterm]:
            for token in rule.right:
                if token.is_term():
                    left = Token(token.value)
                    left.is_guard = True
                    new_rule = Rule(left, [Token(token.value)])
                    if new_rule not in new_rules:
                        new_rules.append(new_rule)
                    token.is_guard = True
    
    new_rules += previous_cfg.rules
    
    return CFG(previous_cfg.start_nterm, new_rules)

## FIRST

In [4]:
@add_method(CFG)
def FIRST(self, tokens, k=1):
    first = {nterm : Set() for nterm in self.nterms}
    
    while True:
        previous_first = first.copy()
        for nterm in first:
            for rule in self[nterm]:
                product = Set([''])
                for token in rule.right:
                    if token.is_term():
                        product *= Set([str(token)])
                    else:
                        product *= first[token]
                first[nterm] += product.first(k)
        
        if previous_first == first:
            break
    
    product = Set([''])
    for token in tokens:
        if token.is_term():
            product *= Set([str(token)])
        else:
            product *= first[token]
    
    return product.first(k)

## FOLLOW

In [5]:
@add_method(CFG)
def FOLLOW(self, nterm, k=1):
    follow = {nterm : Set() for nterm in self.nterms}
    follow[self.start_nterm] = Set(['$'])
    while True:
        previous_follow = follow.copy()
        for rule in self.rules:
            if rule.is_terminal():
                continue
            for partition in rule.partitions():
                follow[partition[0]] += Set.first(
                    self.FIRST(partition[1][1], k) * follow[rule.left],
                    k
                )
        if previous_follow == follow:
            break
    
    return follow[nterm]

## LAST

In [6]:
@add_method(CFG)
def LAST(self, tokens, k=1):
    last = {nterm : Set() for nterm in self.nterms}
    
    while True:
        previous_last = last.copy()
        for nterm in last:
            for rule in self[nterm]:
                product = Set([''])
                for token in rule.right:
                    if token.is_term():
                        product *= Set([str(token)])
                    else:
                        product *= last[token]
                last[nterm] += product.last(k)
        
        if previous_last == last:
            break
    
    product = Set([''])
    for token in tokens:
        if token.is_term():
            product *= Set([str(token)])
        else:
            product *= last[token]
    
    return product.last(k)

## PRECEDE

In [7]:
@add_method(CFG)
def PRECEDE(self, nterm, k=1):
    precede = {nterm : Set() for nterm in self.nterms}
    precede[self.start_nterm] = Set(['^'])
    while True:
        previous_precede = precede.copy()
        for rule in self.rules:
            if rule.is_terminal():
                continue
            for partition in rule.partitions():
                precede[partition[0]] += Set.last(
                    precede[rule.left] * self.LAST(partition[1][0], k),
                    k
                )
        if previous_precede == precede:
            break
    
    return precede[nterm]

## Поиск токенов

**Утверждение** \
Терминал $a$ принадлежит языку нетерминала $T$ $\Leftrightarrow$ $a \in {FIRST}_{2}(T)$

**Следствие** \
Пусть $T$ - регулярный нетерминал. \
$T$ является токеном $\Leftrightarrow {FIRST}_{2}(T) \cap ({FOLLOW}_{1}(T) \cup {PRECEDE}_{1}(T)) = \varnothing$

In [8]:
def tokenize(cfg):
    tokens = set()
    conflicted = set()
    for nterm in cfg.regular_nterms():
        if len(
            cfg.FIRST([nterm], 2)
            .intersection(
                cfg.FOLLOW(nterm, 1) + 
                cfg.PRECEDE(nterm, 1)
            )
        ) == 0:
            tokens.add(nterm)
        else:
            conflicted.add(nterm)
    
    return tokens, conflicted

## Порождение регулярных выражений

Под *заведомо регулярным* нетерминалом будем понимать регулярный нетерминал, найденный до замыкания.

Пусть $G = \langle \Sigma, N, S, P \rangle$ - исходная грамматика, $M \subset N$ - множество регулярных нетерминалов, $M_R \subset M$ - множество заведомо регулярных нетерминалов,  $M_T \subset M$ - множество токенов.

**Алгоритм**
1. Для всех $A \in M_R$ строим грамматику $G_A = \langle \Sigma, N, A, P \rangle$
2. Фильтруем полезные нетерминалы $G_A$
3. Если $G_A$ - леволинейная грамматика, переводим ее в праволинейную
4. Для $A$ порождаем регулярное выражение по $G_A$
5. Для всех $B \in M \setminus M_R$ строим дерево разбора $T$ до первых накачек $\gamma$, таких, что $\exists C \in N: (C \in \gamma \Rightarrow C \in M')$
6. Порождаем регулярное выражение для $B = \displaystyle \sum_{\alpha \in leaves(T)} substitute(\alpha, M') $
7. Для всех $C \in M_T$ возвращаем регулярное выражение

In [9]:
def token_regexes(cfg, tokens):
	lr_nterms = cfg.regular_subset(Rule.is_left_linear)

	rr_nterms = cfg.regular_subset(Rule.is_right_linear)
	
	other_nterms = cfg.regular_nterms().difference(lr_nterms.union(rr_nterms))

	regexes = {}
	for nterm in lr_nterms:
		rg = RG.from_cfg(cfg, nterm)
		rg.to_right_linear()
		regexes[nterm] = rg.to_regex()
	for nterm in rr_nterms:
		rg = RG.from_cfg(cfg, nterm)
		regexes[nterm] = rg.to_regex()
	for nterm in other_nterms:
		tree = PumpTree(cfg, nterm, regexes)
		regexes[nterm] = tree.generate_regex()
	
	result = {}
	for nterm in regexes:
		if nterm in tokens:
			result[nterm] = regexes[nterm]
	
	return result

## Автолексер

In [12]:
def autolex(cfg):
    guarded_cfg = guard(cfg)
    tokens, conflicted = tokenize(guarded_cfg)
    regexes = token_regexes(guarded_cfg, tokens)

    return guarded_cfg, regexes, conflicted

## Тесты

In [14]:
TESTS_DIR = 'tests'

for test_name in os.listdir(TESTS_DIR):
    test = Test(
        test_name.split('.')[0],
        os.path.join(TESTS_DIR, test_name)
    )
    test.execute(autolex)
    Test.add(test)

Test.display_results()